# RoadAI - From PyTorch to Android

This notebook converts the original trained PyTorch model to a .pt model compatible for Android Deployment.

In [20]:
import torch
import numpy as np
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
%matplotlib inline
import torchvision.models as models
from collections import OrderedDict
from torch import nn
from torch import optim

#Load the pretrained vgg19 model
model = models.vgg19(pretrained=True)

# Freeze training for all "features" layers
for param in model.features.parameters():
    param.requires_grad = False
    
#Print the model's architecture
#print (model)    

#Change the classifier's layers - Define my own classifier
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(25088, 100)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(100, 20)),
                          ('relu', nn.ReLU()),
                          ('fc3', nn.Linear(20, 2)),
                          ('relu', nn.ReLU()), 
                          ('output', nn.LogSoftmax(dim=1))
                          ]))

#Change the classifier's layers - Change the classifier section in VGG19
model.classifier = classifier

In [21]:
import torch
from torchvision.models import mobilenet_v2

#model = mobilenet_v2(pretrained=True)
device = torch.device('cpu')
model.load_state_dict(torch.load('roadai_model_pyrtoch.pt', map_location=device))

model.eval()
input_tensor = torch.rand(1,3,224,224)

script_model = torch.jit.trace(model,input_tensor)
script_model.save("roadai_model_android.pt")